In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [104]:
df = pd.read_csv("../data/skylab_instagram_datathon_dataset.csv", sep=";")

df.shape

(704313, 15)

In [105]:
# information about the data before processing it -> run again after processing
df.describe(include='all')

,period,period_end_date,compset_group,compset,business_entity_doing_business_as_name,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,calculation_type,followers,pictures,videos,comments,likes
count,704313,704313,704313,704313,704313,676558,458589,676558,458589,704313,5.792580e+05,695803.000000,684349.000000,6.953430e+05,6.959770e+05
unique,1,455,20,54,706,423,26,401,30,1,NaN,NaN,NaN,NaN,NaN
top,Weekly,2023-07-08,Luxury & Premium & Mainstream,US Softlines Analyst Interest List,All Brands,LVMH Moet Hennessy Louis Vuitton SE,United States of America,LVMH Moet Hennessy Louis Vuitton SE,New York Stock Exchange,Metric Value,NaN,NaN,NaN,NaN,NaN
freq,704313,1635,216241,113744,27755,27576,193509,27576,132071,704313,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.342432e+07,344.272540,61.244426,3.043246e+04,2.163189e+06
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.106975e+08,2777.396873,564.468480,2.766459e+05,1.803193e+07
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.013372e+05,19.000000,2.000000,3.710000e+02,1.727700e+04
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.104144e+06,44.000000,7.000000,1.435000e+03,9.318400e+04
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.479325e+06,122.000000,20.000000,5.872000e+03,4.577470e+05


# Process data

## Data cleaing

### Replace

In [106]:
# READJUST STRINGS

# remove leading and trailing whitespaces and convert to lowercase
df = df.map(lambda x: x.strip().lower() if isinstance(x, str) else x)

# replace all double whitespaces with single whitespaces
df = df.map(lambda x: x.replace("  ", " ") if isinstance(x, str) else x)

In [107]:
# Converting the 'period_end_of_week' column to datetime
df['period_end_date'] = pd.to_datetime(df['period_end_date'])

In [108]:
# REPLACE DOMICILE_COUNTRY_NAME

# replace the china;hong kong with hong kong
df['domicile_country_name'] = df['domicile_country_name'].replace('china;hong kong', 'hong kong')

# remove the sign ";" in column domicile_country_name
df['domicile_country_name'] = df['domicile_country_name'].str.replace(';', '')

# if empty, fill with nan
df['domicile_country_name'] = df['domicile_country_name'].replace('', np.nan)

In [109]:
# REPLACE PRIMARY_EXCHANGE_NAME

# remove the sign ";" in column domicile_country_name
df['primary_exchange_name'] = df['primary_exchange_name'].str.replace(';', '')

# if empty, fill with nan
df['primary_exchange_name'] = df['primary_exchange_name'].replace('', np.nan)

In [110]:
# REPLACE ULTIMATE_PARENT_LEGAL_ENTITY_NAME
df['ultimate_parent_legal_entity_name'] = df['ultimate_parent_legal_entity_name'].replace('Anheuser-Busch;Anheuser-Busch', 'Anheuser-Busch')

### Remove

In [111]:
# Remove "All_Brands" and "Don't Use it"
df = df[df["business_entity_doing_business_as_name"] != "all brands"]
# TODO: Remove "Don't Use it"

In [112]:
# remove unnecessary columns
# Here: 'period', 'calculation_type'
df = df.drop(columns=['period', 'calculation_type'])

In [113]:
# certain entries have exact data except of "compset"
# we want to take the union of all of them

grouping_columns = [col for col in df.columns if col != 'compset']

df.fillna('Group_Null', inplace=True)
result = df.groupby(grouping_columns).agg({'compset': lambda x: set(x)}).reset_index()
df = result
df.replace('Group_Null', np.nan, inplace=True)


# df.describe(include='all')

# to test
#result[(result["business_entity_doing_business_as_name"] == "24S") & (result["period_end_date"] == "2017-05-13")]
#result[(result["business_entity_doing_business_as_name"] == "dd's Discounts") & (result["period_end_date"] == "2023-09-09")]
#result[(result["business_entity_doing_business_as_name"] == "dd's Discounts") & (result["period_end_date"] == "2023-09-16")]

/tmp/ipykernel_14304/591243391.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Group_Null' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('Group_Null', inplace=True)
/tmp/ipykernel_14304/591243391.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('Group_Null', np.nan, inplace=True)


In [114]:
result.shape

(298040, 13)

## Add features

In [115]:
# split the date into year, month, day
df['Year'] = df['period_end_date'].dt.year
df['Month'] = df['period_end_date'].dt.month
df['Day'] = df['period_end_date'].dt.day

In [116]:
# get the difference between the current date and the previous date
df = df.sort_values(by=['business_entity_doing_business_as_name', 'period_end_date'])
df['date_diff_prev'] = df['period_end_date'].diff().dt.days
df['date_diff_prev'] = df['date_diff_prev'].fillna(7)

In [117]:
#df[df['date_diff_prev'] == 0]

In [118]:
# total involvement
df["total_involvement"] = df["comments"] + df["likes"]
df["total_company_activity"] = df["pictures"] + df["videos"]

df["conversion_rate_total"] = df["total_involvement"] / df["followers"]

# COntent type
df["ratio_of_videos"] = df["videos"] / (df["pictures"] + df["videos"])
df["ratio_of_pictures"] = df["pictures"] / (df["pictures"] + df["videos"])

# ASSUMTION: we only like videos / photos from this week
df["likes_per_picture"] = df["likes"] / df["pictures"]
df["likes_per_video"] =   df["likes"] / df["videos"]
df["comments_per_picture"] =  df["comments"] / df["pictures"] 
df["comments_per_video"] =   df["comments"] / df["videos"]

# take care of zeros
df.loc[df["pictures"] + df["videos"] == 0, "ratio_of_videos"] = np.nan
df.loc[df["pictures"] + df["videos"] == 0, "ratio_of_pictures"] = np.nan

df.loc[df["pictures"] == 0, "likes_per_picture"] = np.nan
df.loc[df["videos"] == 0, "likes_per_video"] = np.nan
df.loc[df["pictures"] == 0, "comments_per_picture"] = np.nan
df.loc[df["videos"] == 0, "comments_per_video"] = np.nan

In [121]:
df.describe()

,period_end_date,followers,pictures,videos,comments,likes,Year,Month,Day,date_diff_prev,total_involvement,total_company_activity,conversion_rate_total,ratio_of_videos,ratio_of_pictures,likes_per_picture,likes_per_video,comments_per_picture,comments_per_video
count,298040,2.363130e+05,294176.000000,288438.000000,2.939580e+05,2.942880e+05,298040.000000,298040.000000,298040.000000,298040.000000,2.939580e+05,288326.000000,232915.000000,283921.000000,283921.000000,2.872130e+05,2.468540e+05,286959.000000,2.466890e+05
mean,2019-07-29 07:54:48.170714880,3.795494e+06,119.606086,24.831870,9.528576e+03,6.302697e+05,2019.083328,6.415666,15.666827,0.007821,6.405053e+05,146.521930,0.189906,0.179683,0.820317,7.431102e+03,7.285143e+04,100.068852,9.154792e+02
min,2015-01-03 00:00:00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,2015.000000,1.000000,1.000000,-3178.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,2017-07-08 00:00:00,1.997470e+05,16.000000,2.000000,2.600000e+02,9.426000e+03,2017.000000,4.000000,8.000000,7.000000,1.010425e+04,22.000000,0.044883,0.048077,0.750000,3.268333e+02,1.700130e+03,8.943095,5.057143e+01
50%,2019-08-24 00:00:00,7.834550e+05,39.000000,6.000000,1.100000e+03,5.689150e+04,2019.000000,6.000000,16.000000,7.000000,5.911600e+04,49.000000,0.106937,0.129032,0.870968,1.127333e+03,7.155862e+03,24.549020,1.478571e+02
75%,2021-09-04 00:00:00,2.990059e+06,102.000000,19.000000,4.503000e+03,2.823685e+05,2021.000000,9.000000,23.000000,7.000000,2.905498e+05,126.000000,0.225776,0.250000,0.951923,4.771083e+03,3.582180e+04,72.648843,4.771818e+02
max,2023-09-16 00:00:00,4.301770e+08,10498.000000,1910.000000,3.179575e+06,1.206897e+08,2023.000000,12.000000,31.000000,7.000000,1.209907e+08,11837.000000,17.680733,1.000000,1.000000,1.660860e+06,3.488018e+07,32546.791667,1.218564e+06
std,NaN,1.323278e+07,265.156184,72.078397,5.394751e+04,2.529721e+06,2.456062,3.410693,8.787733,145.318041,2.552247e+06,312.947129,0.281915,0.181286,0.181286,2.342808e+04,3.668371e+05,425.596458,6.294418e+03


In [120]:
sorted(list(df["domicile_country_name"].value_counts().keys()))

df["domicile_country_name"].value_counts()

domicile_country_name
united states of america                                86036
france                                                  26956
switzerland                                             13625
united kingdom of great britain and northern ireland    10884
brazil                                                   9999
japan                                                    8532
hong kong                                                8342
belgium                                                  6725
germany                                                  5248
australia                                                4004
spain                                                    3185
netherlands                                              3140
italy                                                    3033
canada                                                   2656
new zealand                                              1742
sweden                                          